In [0]:
-- DDL con Foreign Keys
CREATE TABLE IF NOT EXISTS VALUTA (
    valuta_id BIGINT PRIMARY KEY,
    codice STRING,
    descrizione STRING,
    simbolo STRING
);

CREATE TABLE IF NOT EXISTS CAMBIO_BUDGET (
    valuta_id BIGINT PRIMARY KEY,
    valore_cambio_eur DECIMAL(10,4),
    FOREIGN KEY (valuta_id) REFERENCES VALUTA(valuta_id)
);

CREATE TABLE IF NOT EXISTS SOCIETA (
    societa_id BIGINT PRIMARY KEY,
    nome STRING,
    codice_fiscale STRING,
    nazione STRING,
    partita_iva STRING
);

CREATE TABLE IF NOT EXISTS CATEGORIA (
    categoria_id BIGINT PRIMARY KEY,
    nome STRING
);

CREATE TABLE IF NOT EXISTS MARCHIO (
    marchio_id BIGINT PRIMARY KEY,
    nome STRING
);

CREATE TABLE IF NOT EXISTS STAGIONE (
    stagione_id BIGINT PRIMARY KEY,
    nome STRING,
    data_inizio DATE,
    data_fine DATE
);

CREATE TABLE IF NOT EXISTS CALENDARIO (
    data DATE PRIMARY KEY,
    mese_fiscale STRING,
    trimestre_fiscale STRING,
    semestre_fiscale STRING,
    anno_fiscale STRING
);

CREATE TABLE IF NOT EXISTS PUNTO_VENDITA (
    punto_vendita_id BIGINT PRIMARY KEY,
    codice STRING,
    nome STRING,
    citta STRING,
    paese STRING,
    formato STRING,
    societa_id BIGINT,
    valuta_id BIGINT,
    FOREIGN KEY (societa_id) REFERENCES SOCIETA(societa_id),
    FOREIGN KEY (valuta_id) REFERENCES VALUTA(valuta_id)
);

CREATE TABLE IF NOT EXISTS CLIENTE (
    cliente_id BIGINT PRIMARY KEY,
    nome STRING,
    genere STRING,
    nazione STRING
);

CREATE TABLE IF NOT EXISTS PRODOTTO (
    prodotto_id BIGINT PRIMARY KEY,
    codice STRING,
    nome STRING,
    prezzo DECIMAL(10,2),
    categoria_id BIGINT,
    marchio_id BIGINT,
    stagione_id BIGINT,
    FOREIGN KEY (categoria_id) REFERENCES CATEGORIA(categoria_id),
    FOREIGN KEY (marchio_id) REFERENCES MARCHIO(marchio_id),
    FOREIGN KEY (stagione_id) REFERENCES STAGIONE(stagione_id)
);

-- =============================================================
-- DDL FACT TABLES with Foreign Keys
-- Catalog/Schema are assumed already selected via USE statements
-- =============================================================

USE CATALOG fashion_catalog;
USE SCHEMA fashion_synthetic_2;

-- SCONTRINO (Header)
CREATE TABLE IF NOT EXISTS SCONTRINO (
    scontrino_id BIGINT PRIMARY KEY,
    cliente_id BIGINT NOT NULL,
    data_scontrino DATE NOT NULL,
    valuta_id BIGINT NOT NULL,
    punto_vendita_id BIGINT NOT NULL,
    -- Foreign Keys
    FOREIGN KEY (cliente_id) REFERENCES CLIENTE(cliente_id),
    FOREIGN KEY (valuta_id) REFERENCES VALUTA(valuta_id),
    FOREIGN KEY (punto_vendita_id) REFERENCES PUNTO_VENDITA(punto_vendita_id),
    FOREIGN KEY (data_scontrino) REFERENCES CALENDARIO(data)
);

-- DETTAGLIO_SCONTRINO (Lines)
CREATE TABLE IF NOT EXISTS DETTAGLIO_SCONTRINO (
    scontrino_id BIGINT NOT NULL,
    numero_riga INT NOT NULL,
    prodotto_id BIGINT NOT NULL,
    quantita INT NOT NULL,
    prezzo_unitario_locale DECIMAL(10,2) NOT NULL,
    -- Primary Key
    PRIMARY KEY (scontrino_id, numero_riga),
    -- Foreign Keys
    FOREIGN KEY (scontrino_id) REFERENCES SCONTRINO(scontrino_id),
    FOREIGN KEY (prodotto_id) REFERENCES PRODOTTO(prodotto_id)
);

-- BUDGET_SCONTRINO (Daily Budget per Store/Currency)
CREATE TABLE IF NOT EXISTS BUDGET_SCONTRINO (
    data DATE NOT NULL,
    punto_vendita_id BIGINT NOT NULL,
    valuta_id BIGINT NOT NULL,
    budget_vendite DECIMAL(12,2) NOT NULL,
    -- Primary Key
    PRIMARY KEY (data, punto_vendita_id, valuta_id),
    -- Foreign Keys
    FOREIGN KEY (data) REFERENCES CALENDARIO(data),
    FOREIGN KEY (punto_vendita_id) REFERENCES PUNTO_VENDITA(punto_vendita_id),
    FOREIGN KEY (valuta_id) REFERENCES VALUTA(valuta_id)
);

